Третий файл, который называется sample_supplier.csv, содержит в себе
прайс лист. который необходимо сравнить с тем, который вы получили в
предыдущем задании. Позиции считаются одинаковыми если у них
одинаковый part_number и manufacture
r.
Сравнение должно показать итоговому пользователю в каком из прайс
листов лучше цены по пересекающимся позициям. Стоит также учесть, что
итоговый вариант будет открыт в Excel, поэтому необходимо предпринять
меры по его правильному форматиро
ванию.
Метод сравнения: SQL запрос.

In [1]:
import pandas as pd
import sqlite3
import numpy as np

In [2]:
sample_supplier = pd.read_csv('sample_supplier.csv', sep='\t')
filtered_data = pd.read_csv('filtered_data.csv', sep='\t')

In [3]:
conn = sqlite3.connect(':memory:')

sample_supplier.to_sql('sample_supplier', conn, index=False, if_exists='replace')
filtered_data.to_sql('filtered_data', conn, index=False, if_exists='replace')

query = '''
    SELECT
    a.part_number,
    a.manufacturer,
    a.price AS price_sample_supplier,
    b.price AS price_filtered_data,
    CASE
        WHEN a.price < b.price THEN 'sample_supplier'
        WHEN a.price > b.price THEN 'filtered_data'
        ELSE 'equal'
    END AS better_price
FROM
    sample_supplier a
JOIN
    filtered_data b
ON
    a.part_number = b.part_number
    AND a.manufacturer = b.manufacturer;
'''

result = pd.read_sql_query(query, conn)

In [4]:
result

,part_number,manufacturer,price_sample_supplier,price_filtered_data,better_price
0,226 920,LINEX,692.93,28.20,filtered_data
1,226 920,LINEX,692.93,28.20,filtered_data
2,226 920,LINEX,692.93,28.20,filtered_data
3,226 920,LINEX,692.93,28.20,filtered_data
4,039 042,SUNSTAR,531.74,50.51,filtered_data
...,...,...,...,...,...
1076380,023 109,ZWM CZAJA,623.44,5.18,filtered_data
1076381,023 109,ZWM CZAJA,623.44,5.18,filtered_data
1076382,023 109,ZWM CZAJA,623.44,5.18,filtered_data
1076383,023 109,ZWM CZAJA,623.44,5.18,filtered_data


In [5]:
result_chunked = np.array_split(result, len(result) // 1048576 + 1)
with pd.ExcelWriter('price_comparison_result.xlsx') as writer:
    for i, df_chunk in enumerate(result_chunked):
        df_chunk.to_excel(writer, sheet_name=f'Sheet_{i+1}', index=False)

C:\Users\ArseniBoika\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [6]:
conn.close()